New approach: grab all votes every day. Then store the query info in a db.
```
house_members:
    member_id:
        [{ bill_id, date, roll_call, session, chamber, vote_position}]
```

In [1]:
from utils import RenderJSON, fetch_all_items

import streamlit as st
from deta import Deta
from congress import Congress
# from datetime import 

SENATE = 'senate'
HOUSE = 'house'

DETA_API_KEY = st.secrets["DETA_API_KEY"];
PROPUBLICA_API_KEY = st.secrets["PROPUBLICA_API_KEY"];

congress = Congress(PROPUBLICA_API_KEY)
deta = Deta(DETA_API_KEY)

- For every day:
    - get all votes
    - for every vote:
        - get details
        - for every position:
            - member_id
            - bill_id
            - date
            - time
            - position
            - chamber
            - rollcall_num
            - session
            - congress

In [20]:
votes_by_day = congress.votes.by_date(HOUSE, "2022-12-21")
print(len(votes_by_day["votes"]))
RenderJSON(votes_by_day)

8


In [21]:
vote_details = congress.votes.get(
    chamber      = votes_by_day["votes"][0]['chamber'],
    rollcall_num = votes_by_day["votes"][0]['roll_call'],
    session      = votes_by_day["votes"][0]['session'],
    congress     = votes_by_day["votes"][0]['congress'],
)
RenderJSON(vote_details)

In [25]:
votes_by_day = congress.votes.by_date(SENATE, "2022-12-21")
print(len(votes_by_day["votes"]))
RenderJSON(votes_by_day)

2


In [26]:
vote_details = congress.votes.get(
    chamber      = votes_by_day["votes"][0]['chamber'],
    rollcall_num = votes_by_day["votes"][0]['roll_call'],
    session      = votes_by_day["votes"][0]['session'],
    congress     = votes_by_day["votes"][0]['congress'],
)
RenderJSON(vote_details)

In [17]:
# def process_vote_on_date(votes_by_day):
#     results = {}

#     print("number of votes on this day:", len(votes_by_day["votes"]))

#     # for vote_on_day in votes_by_day["votes"]:


#     # MUST MOVE STUFF BELOW
#     chamber      = votes_by_day["votes"][0]['chamber']
#     rollcall_num = votes_by_day["votes"][0]['roll_call']
#     session      = votes_by_day["votes"][0]['session']
#     congress_n     = votes_by_day["votes"][0]['congress']


#     vote_details = congress.votes.get(
#         chamber      = chamber,
#         rollcall_num = rollcall_num,
#         session      = session,
#         congress     = congress_n,
#     )
#     if "votes" in vote_details:



#         for v in vote_details["votes"]:

#             if "vote" in vote_details["votes"]:
#                 date = vote_details["votes"]["vote"]["date"]
#                 time = vote_details["votes"]["vote"]["time"]
#                 if "bill" in vote_details["votes"]["vote"]:
#                     bill_id = None
#                     if "bill_id" in vote_details["votes"]["vote"]["bill"]:
#                         bill_id = vote_details["votes"]["vote"]["bill"]["bill_id"]
#                 if "positions" in vote_details["votes"]["vote"]:
#                     positions = vote_details["votes"]["vote"]["positions"]
#                     for position in positions:
#                         member_id     = position["member_id"]
#                         vote_position = position["vote_position"]
#                         if member_id not in results:
#                             results[member_id] = []
#                         results[member_id].append({
#                             "bill_id": bill_id,
#                             "date": date,
#                             "time": time,
#                             "vote_position": vote_position,
#                             "chamber": chamber,
#                             "rollcall_num": rollcall_num,
#                             "session": session,
#                             "congress": congress_n,
#                         })
#     return results

# results = process_vote_on_date(votes_by_day)

# RenderJSON(results)

In [36]:
def process_vote_on_date(votes_by_day, verbose=False):
    results = {}

    if verbose:
        print("number of votes on this day:", len(votes_by_day["votes"]))

    for vote_on_day in votes_by_day["votes"]:
        chamber        = vote_on_day['chamber']
        rollcall_num   = vote_on_day['roll_call']
        session        = vote_on_day['session']
        congress_n     = vote_on_day['congress']

        vote_details = congress.votes.get(
            chamber      = chamber,
            rollcall_num = rollcall_num,
            session      = session,
            congress     = congress_n,
        )
        
        if "votes" in vote_details:
            if verbose:
                print("number of votes in vote_details:", len(vote_details["votes"]))
                
            if "vote" in vote_details["votes"]:
                vote_result = vote_details["votes"]["vote"]["result"]
                date        = vote_details["votes"]["vote"]["date"]
                time        = vote_details["votes"]["vote"]["time"]
                if "bill" in vote_details["votes"]["vote"]:
                    bill_id = None
                    if "bill_id" in vote_details["votes"]["vote"]["bill"]:
                        bill_id = vote_details["votes"]["vote"]["bill"]["bill_id"]
                if "positions" in vote_details["votes"]["vote"]:
                    positions = vote_details["votes"]["vote"]["positions"]
                    for position in positions:
                        member_id     = position["member_id"]
                        vote_position = position["vote_position"]
                        if member_id not in results:
                            results[member_id] = {}
                        if vote_result not in results[member_id]:
                            results[member_id][vote_result] = []
                        results[member_id][vote_result].append({
                            "bill_id": bill_id,
                            "date": date,
                            "time": time,
                            "vote_position": vote_position,
                            "chamber": chamber,
                            "rollcall_num": rollcall_num,
                            "session": session,
                            "congress": congress_n,
                        })
    return results

results = process_vote_on_date(votes_by_day)

RenderJSON(results)

# Putting it all together

In [37]:
house_votes = []
senate_votes = []

from datetime import datetime, timedelta

selected_day = datetime.today()
past_limit = datetime(2023, 3, 1, 0, 0, 0) # Jan 1st, 2010
delta = timedelta(days=1)

while selected_day > past_limit:
    date_str = selected_day.strftime('%Y-%m-%d')
    # print(date_str)
    house_votes_by_day = congress.votes.by_date(HOUSE, date_str)
    house_votes.extend(house_votes_by_day['votes'])
    
    senate_votes_by_day = congress.votes.by_date(SENATE, date_str)
    senate_votes.extend(senate_votes_by_day['votes'])

    selected_day -= delta

In [32]:
RenderJSON(house_votes)

In [38]:
from datetime import datetime, timedelta

def get_votes_between(chamber, from_dt, to_dt = datetime.today()):
    assert(to_dt > from_dt)

    total_votes = []

    selected_day = to_dt
    delta = timedelta(days=1)

    while selected_day > from_dt:
        date_str = selected_day.strftime('%Y-%m-%d')
        # print(date_str)
        votes_by_day = congress.votes.by_date(chamber, date_str)
        total_votes.extend(votes_by_day['votes'])
    
        selected_day -= delta

    return total_votes

house_votes = get_votes_between(HOUSE, datetime(2023, 3, 1, 0, 0, 0))
RenderJSON(house_votes)

In [41]:
all_results = process_vote_on_date({'votes': house_votes})
RenderJSON(all_results)

---
# Here it is!

In [43]:
def process_votes_to_member_positions(votes_list, verbose=False):
    results = {}

    if verbose:
        print("number of votes:", len(votes_list))

    for vote_in_list in votes_list:
        chamber        = vote_in_list['chamber']
        rollcall_num   = vote_in_list['roll_call']
        session        = vote_in_list['session']
        congress_n     = vote_in_list['congress']

        vote_details = congress.votes.get(
            chamber      = chamber,
            rollcall_num = rollcall_num,
            session      = session,
            congress     = congress_n,
        )
        
        if "votes" in vote_details:
            if verbose:
                print("number of votes in vote_details:", len(vote_details["votes"]))
                
            if "vote" in vote_details["votes"]:
                vote_result = vote_details["votes"]["vote"]["result"]
                date        = vote_details["votes"]["vote"]["date"]
                time        = vote_details["votes"]["vote"]["time"]
                if "bill" in vote_details["votes"]["vote"]:
                    bill_id = None
                    if "bill_id" in vote_details["votes"]["vote"]["bill"]:
                        bill_id = vote_details["votes"]["vote"]["bill"]["bill_id"]
                if "positions" in vote_details["votes"]["vote"]:
                    positions = vote_details["votes"]["vote"]["positions"]
                    for position in positions:
                        member_id     = position["member_id"]
                        vote_position = position["vote_position"]
                        if member_id not in results:
                            results[member_id] = {}
                        if vote_result not in results[member_id]:
                            results[member_id][vote_result] = []
                        results[member_id][vote_result].append({
                            "bill_id": bill_id,
                            "date": date,
                            "time": time,
                            "vote_position": vote_position,
                            "chamber": chamber,
                            "rollcall_num": rollcall_num,
                            "session": session,
                            "congress": congress_n,
                        })
    return results

all_results = process_votes_to_member_positions(house_votes)
RenderJSON(all_results)